# Data Preparation for Collaborative Filtering

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
df = pd.read_csv('Comments.csv')

In [3]:
df.head(3)

,id,course_id,rate,date,display_name,comment
0,88962892,3173036,1.0,2021-06-29T18:54:25-07:00,Rahul,I think a beginner needs more than you think.\...
1,125535470,4913148,5.0,2022-10-07T11:17:41-07:00,Marlo,Aviva is such a natural teacher and healer/hea...
2,68767147,3178386,3.5,2020-10-19T06:35:37-07:00,Yamila Andrea,Muy buena la introducción para entender la bas...


Only needed information for collaborative filtering is user, course and the rate.

In [4]:
reviews_df = df[['display_name', 'course_id', 'rate']]

In [5]:
reviews_df.head(3)

,display_name,course_id,rate
0,Rahul,3173036,1.0
1,Marlo,4913148,5.0
2,Yamila Andrea,3178386,3.5


Information about the ratings.

In [6]:
reviews_df.describe().round(2)["rate"]

count    9411727.00
mean           4.51
std            0.86
min            0.50
25%            4.00
50%            5.00
75%            5.00
max            5.00
Name: rate, dtype: float64

In [7]:
reviews_df.isnull().sum()

display_name    75362
course_id           0
rate                0
dtype: int64

Clearing the anonymous users and duplicate entries if exist.

In [8]:
review_df = reviews_df.dropna()
review_df = reviews_df.drop_duplicates()

Most of the courses reviewed just once. We selected popular courses in order increase the relevancy of the recommendations and decrease computational load.

In [9]:
courses = review_df['course_id'].value_counts()
courses

567828     23031
793796     21477
914296     18245
1565838    17786
625204     17542
           ...  
3314984        1
4139034        1
4375656        1
2172942        1
3173036        1
Name: course_id, Length: 162995, dtype: int64

In [10]:
print("Number of courses reviewed more than 500 times:", np.sum(courses > 500))
print("Number of courses reviewed once:", np.sum(courses == 1))

Number of courses reviewed more than 500 times: 2549
Number of courses reviewed once: 20845


In [11]:
# change according to the limitations
c_filter = courses[courses>500]
review_df = review_df.loc[review_df['course_id'].isin(c_filter.axes[0])]
review_df

,display_name,course_id,rate
2720695,Larry,1055720,5.0
2720696,Carley,1055720,5.0
2720697,Abby,1055720,4.0
2720698,Adolphus,1055720,5.0
2720699,Vivienne,1055720,5.0
...,...,...,...
9411719,Sathyanarayanan,567828,5.0
9411723,Rao Saurabh,567828,5.0
9411724,Raveesh,567828,5.0
9411725,Wendell,567828,5.0


Since the dataset does not provide an identification for users, we decided check and select according to display names. Most of the reviews share the same display name. We decided to select reviews have the same display name and between some threshold (5, 10) because it is unlikely to one person use purchase more than 10-15 courses and finishes them and review them.

In [12]:
reviewer_names = review_df['display_name'].value_counts()
reviewer_names

Anonymized         9524
David              6724
Daniel             5899
Michael            5684
John               5090
                   ... 
Gabriel Taborda       1
Joceli Miguel         1
Wesllen Santos        1
Jonatha Rihan         1
Rao Saurabh           1
Name: display_name, Length: 858473, dtype: int64

In [13]:
# change according to the limitations
reviewer_filter = reviewer_names[(reviewer_names>4) & (reviewer_names<10)]
review_df = review_df.loc[review_df['display_name'].isin(reviewer_filter.axes[0])]
review_df

,display_name,course_id,rate
2720700,Herminia,1055720,5.0
2720706,Allene,1055720,5.0
2720716,Jacynthe,1055720,5.0
2720731,Norval,1055720,5.0
2720733,Dovie,1055720,5.0
...,...,...,...
9411623,Jacari,567828,5.0
9411631,Linet,567828,5.0
9411656,Drishtant,567828,5.0
9411667,Claro,567828,5.0


After filtering the users, some of the users seemed to reviewed one course more than once (different users). We dropped these users.

In [14]:
groupby_reviewer = review_df.groupby(["display_name"])
delete = []
for user in groupby_reviewer.groups.keys():
     if groupby_reviewer.get_group(user).nunique()["course_id"] != groupby_reviewer.get_group(user).count()["course_id"]:
        delete.append(user)

review_df = review_df.loc[~review_df['display_name'].isin(delete)]
review_df

,display_name,course_id,rate
2720716,Jacynthe,1055720,5.0
2720731,Norval,1055720,5.0
2720758,Jany,1055720,4.0
2720764,Delmer,1055720,5.0
2720799,Merl,1055720,5.0
...,...,...,...
9411623,Jacari,567828,5.0
9411631,Linet,567828,5.0
9411656,Drishtant,567828,5.0
9411667,Claro,567828,5.0


State of the data after filtering operations.

In [15]:
review_df['display_name'].value_counts()

Jyoshna         9
Nikkie          9
Mulham          9
Shian           9
Liwen           9
               ..
Oyedele         5
Anthony A       5
Ana Salvador    5
Praween         5
ASHWARYA        5
Name: display_name, Length: 38295, dtype: int64

In [16]:
review_df['course_id'].nunique()

2549

We set ids to users to make it easy to work on.

In [17]:
le = LabelEncoder()
reviewer_id = le.fit_transform(review_df["display_name"])
review_df.insert(0, "user_id", reviewer_id, True)
review_df

,user_id,display_name,course_id,rate
2720716,14177,Jacynthe,1055720,5.0
2720731,24654,Norval,1055720,5.0
2720758,14484,Jany,1055720,4.0
2720764,7689,Delmer,1055720,5.0
2720799,22004,Merl,1055720,5.0
...,...,...,...,...
9411623,14118,Jacari,567828,5.0
9411631,19119,Linet,567828,5.0
9411656,8598,Drishtant,567828,5.0
9411667,6454,Claro,567828,5.0


In [18]:
review_df.to_csv("rates.csv", index=False)

Finally we filtered the course data with the courses we left with after all these operations.

In [19]:
reviewed_courses = review_df["course_id"].unique()
courses = pd.read_csv("Course_info.csv")
courses = courses[["id", "title", "category", "course_url"]]
courses = courses[courses["id"].isin(reviewed_courses)]
courses["id"] = courses["id"].astype(np.int64)
courses.to_csv("courses.csv", index=False)